# Tratamiento de Datos Masivos  - Práctica 2 - Solución
### Rafael Caballero - Facultad de Informática - UCM

In [2]:
# importante, debes tener al menos la versión 3.7 de Pymongo
import pymongo
pymongo.version

'3.7.2'

En esta práctica vamos a acceder desde Python a Mongo, tanto para hacer operaciones de modificación como consultas básicas. Las siguientes instrucciones están pensadas para ser utilizadas en el entorno **Windows** de los laboratorios

### Preparación

* En primer lugar necesitamos arrancar el servidor de Mongo. Para ello 
    * Creamos una carpeta nueva en un lugar donde tengamos derechos de escritura (por ejemplo c:\hlocal\tdm\data). Importante: si ya existe la carpeta, borrarla primero.
    * Abrimos una consola de Mongo y  tecleamos: ```mongod -dbpath c:\hlocal\tdm\data ```. Si todo va bien ya tenemos el servidor arrancado. Si no funciona avisad al profesor, que a su vez avisará al técnico, que a su vez...

* El segundo paso es importar la base de datos. Para ello copiamos el fichero comments.json y el fichero comunidades.csv a una carpeta, digamos c:\hlocal\tdm, abrimos otra consola de Mongo y tecleamos
```[Python]
mongoimport -d lv -c comments --file c:\hlocal\tdm\comments.json
```

Nos debe decir que se han importado 164642 documentos

* Finalmente ejecutaremos el siguiente código para crear el acceso a la base de datos y a las colecciones

In [3]:
import pymongo  # la conexión con mongo
from pprint import pprint
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.lv
comments = db.comments
comunidades = db.comunidades # esta aun no existe, la crearemos más adelante en la la práctica
print(comments.count_documents({})) # debe mostrar 164642

164642


### Ejercicios 

**1.-**  Número de comentarios escritor por el autor con 'authorNick' igual a 'UROBOROS3'.
*Nota* : Para contar documentos en Pymongo usar la función comentarios.count_documents(Filtro) con *Filtro* el documento que se usa como filtro en find.

Respuesta esperada: 496

In [4]:
# Solución
comments.count_documents({'content.authorNick':'UROBOROS3'})

496

**2.-** Número de comentarios a los que ha dado like (likedBy) el autor con 'authorNick' igual a 'UROBOROS3'

Respuesta esperada: 282


In [5]:
# Solución
comments.count_documents({'content.likedBy':'UROBOROS3'})

282

**3.-**  Número de comentarios publicados (clave 'createdAt') entre el 12 y el 13 de octubre de 2018.

*Nota* Por ejemplo d = datetime.strptime('2018-10-12','%Y-%m-%d')  devuelve en d el valor del 12 de octubre en un formato compatible con MongoDB

Salida esperada: 20960

In [6]:
from datetime import datetime
# Solución
comments.count_documents({'createdAt':
                          {'$gte':datetime.strptime('2018-10-12','%Y-%m-%d'), '$lte':datetime.strptime('2018-10-13','%Y-%m-%d')} })

20960

** 4.- ** Número de comentarios que han gustado al menos a un usuario (contienen clave 'likedBy').

Salida esperada: 80455

In [7]:
# Solución
comments.count_documents({'content.likedBy':{'$exists':True}})

80455

** 5.- ** Autor (authorNick) del usuario que ha emitido el comentario 

```[JSON]
"_id" : {
                "collection" : "238449142",
                "id" : "811961925"
        }
```
Salida esperada: ```{'content': {'authorNick': '9- Railroad'}}```

In [8]:
# Solución
comments.find_one({"_id" : {
                "collection" : "238449142",
                "id" : "811961925"
        }},{'content.authorNick':1,'_id':0})

{'content': {'authorNick': '9- Railroad'}}

** 6.- ** *(5 puntos)* Consideramos ahora el fichero comunidades.csv que contiene el identificador de cada usuario (campo 1) y la comunidad a la que pertenece (campo 3, 0 no independentista, 1 indenpendentista ).

Queremos añadir a la colección *comunidades* la misma información de *comentarios*, pero añadiendo un campo *comunidad* que indique la comunidad a la que pertenece el usuario que ha emitido el comentario.

In [9]:
comunidades.drop() # borramos lo ya hecho
result = comments.create_index([('content.authorId', pymongo.ASCENDING)],
                                   unique=False)

import csv
path = 'c:/hlocal/tdm/comunidades.csv'
with open(path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        for c in comments.find({'content.authorId':row[0]}):
             c['content']['comunidad'] = row[2]
             comunidades.insert_one(c)



In [10]:
print(comunidades.count_documents({'content.comunidad':'0'}))
print(comunidades.count_documents({'content.comunidad':'1'}))

95775
25544
